In [2]:
from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.metrics import make_scorer
import pandas as pd


recipesdf = pd.read_csv("recipes.csv")
reviewsdf = pd.read_csv("reviews.csv")
reviewsdf = reviewsdf.head(100000)


In [3]:
col_one_list = reviewsdf['AuthorId'].tolist()
col_two_list = reviewsdf['RecipeId'].tolist()
col_three_list = reviewsdf['Rating'].tolist()

ratings_dict = {
    "item": col_two_list,
    "user": col_one_list,
    "rating": col_three_list,
}

df = pd.DataFrame(ratings_dict)

reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[["user", "item", "rating"]], reader)
trainingSet = data.build_full_trainset()
trainset, testset = train_test_split(data, test_size=0.25)


In [4]:
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 1.0435


1.0435206654963163

In [5]:
# knnbasic_tuned = KNNBasic(sim_options={'name': 'cosine', 'user_based': True,'min_support': True, 'min_k': 2,})
# knnbasic_tuned.fit(trainset)
# knnbpreds = knnbasic_tuned.test(testset)

# accuracy.rmse(knnbpreds)
# accuracy.mae(knnbpreds)

In [6]:
# knnbaseline_tuned = KNNBaseline(sim_options={'name': 'cosine', 'user_based': True, 'min_support': True, 'min_k': 2,})
# knnbaseline_tuned.fit(trainset)
# knnbpreds = knnbaseline_tuned.test(testset)

# accuracy.rmse(knnbpreds)
# accuracy.mae(knnbpreds)

In [7]:
# # To use item-based cosine similarity
# sim_options = {
#     "name": "cosine",
#     "user_based": False,  # Compute  similarities between items
# }
# algo = KNNWithMeans(sim_options=sim_options)
# algo.fit(trainingSet)

# prediction = algo.predict('1', 70)
# prediction.est

In [8]:
# sim_options = {
#     "name": ["msd", "cosine"],
#     "min_support": [3, 4, 5],
#     "user_based": [False, True],
# }

# param_grid = {"sim_options": sim_options}

# gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
# gs.fit(data)

# print(gs.best_score)
# print(gs.best_params)


In [9]:
param_grid = {
    "n_epochs": [20,50,100],
    "lr_all":  [0.01,0.02,0.03],
    "reg_all": [0.2,0.1,0.05]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5, refit = True)

gs.fit(data)

print(gs.best_score)
print(gs.best_params)

{'rmse': 1.0379689355371358, 'mae': 0.6200058703029125}
{'rmse': {'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.1}, 'mae': {'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.1}}


In [11]:
prediction = gs.predict(41, 48730)
prediction.est

4.50464